Quoting: https://github.com/romanmichaelpaolucci/Quant-Guild-Library/blob/main/2025%20Video%20Lectures/2.%20Control%20Variates%20for%20Variance%20Reduction/Control%20Variates%20for%20Variance%20Reduction.ipynb
